In [ ]:
!pip install -q -U transformers

In [ ]:
import subprocess, gc, emoji, re

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras as tfk
from transformers import XLMRobertaTokenizer, XLMRobertaConfig, TFXLMRobertaModel, TFXLMRobertaForMaskedLM

In [ ]:
def run_command(command):
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    return process.communicate()

## Dataset

### Load dataset

In [ ]:
run_command("gsutil rm -r product-translation-dataset")
run_command("gsutil cp -r gs://shopee-title-translation/product-translation-dataset .")
run_command("gsutil cp gs://shopee-title-translation/uniq_tokens_masks.csv .")

In [ ]:
df_all = pd.read_csv('product-translation-dataset/all_clean.csv')
df_all['product_title'] = df_all['product_title'].str.lower()
df_all.head()

## Model

### Initialize tokenizer

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained('jplu/tf-xlm-roberta-base')
special_tokens = [tokenizer.bos_token_id, 
                  tokenizer.eos_token_id, 
                  tokenizer.sep_token_id, 
                  tokenizer.pad_token_id, 
                  tokenizer.cls_token_id,
                  tokenizer.mask_token_id]
special_tokens

In [ ]:
mask = pd.read_csv('uniq_tokens_masks.csv')['tcn_en_mask'].to_numpy()
for special_token in special_tokens:
    mask[special_token] = 1
enc_mapping = {}
dec_mapping = {}
for i in range(len(mask)):
    if mask[i] == 0:  continue
    enc_mapping[i] = len(enc_mapping)
    dec_mapping[len(dec_mapping)] = i

In [ ]:
enc_table = tf.lookup.StaticVocabularyTable(
            tf.lookup.KeyValueTensorInitializer(
                list(enc_mapping.keys()),
                list(enc_mapping.values()),
                key_dtype=tf.int64,
                value_dtype=tf.int64,
            ),
            num_oov_buckets=1,
        )

In [ ]:
dec_table = tf.lookup.StaticVocabularyTable(
            tf.lookup.KeyValueTensorInitializer(
                list(dec_mapping.keys()),
                list(dec_mapping.values()),
                key_dtype=tf.int64,
                value_dtype=tf.int64,
            ),
            num_oov_buckets=1,
        )

In [ ]:
class CustomTokenizer(XLMRobertaTokenizer):
    def __init__(self, enc_table, dec_table, **kwargs):
        self.enc_table = enc_table
        self.dec_table = dec_table
        super().__init__(**kwargs)
    
    def __call__(self, text, **kwargs):
        res = super().__call__(text, **kwargs)
        res['input_ids'] = tf.cast(enc_table.lookup(tf.cast(res['input_ids'], tf.int64)), tf.int32)
        return res
    def decode(self, token_ids, **kwargs):
        token_ids = tf.cast(dec_table.lookup(tf.cast(token_ids, tf.int64)), tf.int32)
        return super().decode(token_ids, **kwargs)

tokenizer = CustomTokenizer.from_pretrained('jplu/tf-xlm-roberta-base', enc_table=enc_table, dec_table=dec_table)

In [ ]:
def calc_tokens(sentences, mask_prob=0.15, mask_id=250001):
    res = tokenizer(sentences,
                    max_length=64,
                    truncation=True,
                    #padding='max_length',
                    padding=True,
                    return_tensors='tf',
                    return_attention_mask=True,
                    return_special_tokens_mask=True)
    input_tokens        = res['input_ids']
    attention_mask      = res['attention_mask']
    special_tokens_mask = res['special_tokens_mask']
    
    mask = tf.cast(tf.random.uniform(shape=tf.shape(input_tokens)) < mask_prob, 'int32') * (1 - special_tokens_mask)
    
    masked_input_tokens = input_tokens * (1-mask) + mask_id * mask
    label_input_tokens  = input_tokens * mask + -100 * (1-mask)
    return masked_input_tokens, label_input_tokens, attention_mask

def generate_data(df_train, batch_size, mask_prob=0.3):
    while True:
        df_train = df_train.sample(frac=1.0)
        for i in range(0, len(df_train)-batch_size, batch_size):
            X, Y, attention_mask = calc_tokens(list(df_train['product_title'].iloc[i:i+batch_size].to_numpy()), mask_prob, enc_mapping[250001])
            yield {'input_ids': X,
                   'labels':Y,
                   'attention_mask': attention_mask}, Y

In [ ]:
X_test, Y_test = next(generate_data(df_all, batch_size=3, mask_prob=0.15))

In [ ]:
[tokenizer.decode(X_test['input_ids'][i]) for i in range(3)]

### Initialize configuration

In [ ]:
config = XLMRobertaConfig(vocab_size=44512,
                          hidden_size=256,
                          num_hidden_layers=8,
                          num_attention_heads=4,
                          intermediate_size=1024,
                          hidden_act='gelu',
                          hidden_dropout_prob=0.1,
                          attention_probs_dropout_prob=0.1,
                          max_position_embeddings=68,
                          type_vocab_size=2,
                          initializer_range=0.02,
                          layer_norm_eps=1e-12,
                          gradient_checkpointing=False
                         )

### Build model

In [ ]:
input_ids      = tfk.layers.Input(shape=(None,), name='input_ids', dtype='int32')
attention_mask = tfk.layers.Input(shape=(None,), name='attention_mask', dtype='int32')
x = TFXLMRobertaForMaskedLM(config)([input_ids, attention_mask])[0]
model = tfk.models.Model(inputs=[input_ids, attention_mask], outputs=x)
model.summary()

In [ ]:
tfk.utils.plot_model(model, show_shapes=True, dpi=300)

In [ ]:
def mlm_loss(labels, logits):
    loss_fn = tfk.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE #SUM
    )
    # make sure only labels that are not equal to -100
    # are taken into account as loss
    active_loss = tf.reshape(labels, (-1,)) != -100
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, tf.shape(logits)[2])), active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
    return loss_fn(labels, reduced_logits)

### Train model

In [ ]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.
    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.
    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.
    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(tfk.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.
    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = tfk.backend.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        tfk.backend.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=1e-4,
                                        total_steps=24*120*200,  # 8 hours * 120 epochs/hour * 200 steps/epoch
                                        warmup_learning_rate=1e-7,
                                        warmup_steps=200,
                                        hold_base_rate_steps=200,
                                        global_step_init=267213
                                       )
            
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.2,
    patience=90,
    min_lr=1e-7
)

early_stop = tfk.callbacks.EarlyStopping(
    monitor='loss',
    patience=180,
    restore_best_weights=True
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir="logs",
    histogram_freq=0,
    update_freq=100,
    write_graph=False,
    profile_batch = 0
)

In [ ]:
run_command("gsutil rm -r tcn-en-model-weights-2")
run_command("gsutil cp -r gs://shopee-title-translation/models/tcn-en-model-weights-2 .")
model.load_weights('tcn-en-model-weights-2/tcn_to_en_model')

In [ ]:
model.compile(optimizer=tfk.optimizers.Adam(),
              loss=mlm_loss)

In [ ]:
model.save_weights('tcn-en-model-weights-2/tcn_to_en_model')
run_command("gsutil rm -r gs://shopee-title-translation/models/tcn-en-model-weights-2")
run_command("gsutil cp -r tcn-en-model-weights-2 gs://shopee-title-translation/models")

In [ ]:
for v in range(0, 50+1):
    print("---------------- {} ----------------".format(v))
    model.fit(generate_data(df_all, batch_size=256, mask_prob=0.15),
              steps_per_epoch=400,
              epochs=30,
              callbacks=[
                reduce_lr,
                early_stop,
                warm_up_lr,
                #tensorboard_callback
              ])
    model.save_weights('tcn-en-model-weights-2/tcn_to_en_model')
    run_command("gsutil rm -r gs://shopee-title-translation/models/tcn-en-model-weights-2")
    run_command("gsutil cp -r tcn-en-model-weights-2 gs://shopee-title-translation/models")

In [ ]:
warm_up_lr.global_step

In [ ]:
X, Y = next(generate_data(df_all, 2, 0.15))
X

In [ ]:
tokenizer.decode(X['input_ids'][0])

In [ ]:
res = model.predict(X)

In [ ]:
res.argmax(axis=2)[0]

In [ ]:
          0,  3215,  1336,   479, 40612, 27595, 44509,  5545, 13007,
      44509,  9796, 27709,  9796, 21930, 41102,  9796,   819, 44509,
       6723,   777,  5545, 26731, 33191,  9796,     6, 44509,  2288,
       2288,     2

In [ ]:
tokenizer.decode(res.argmax(axis=2)[0])